In [122]:
CLIENT_ID = 'ED51L2XHFB1IIKHKRCX3QDY2EXUOZ25SHHQV4JSTHFRECLH4' # your Foursquare ID
CLIENT_SECRET = 'T2BXC1HLDCLIY02VDN3NRTXK3DIAZWZY0KO1BJC0AV3FOPLA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
!pip install folium 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import requests
import urllib.request
import time
from bs4 import BeautifulSoup

from urllib.request import urlopen


Your credentials:
CLIENT_ID: SECRET
CLIENT_SECRET:SECRET
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


## We scrape Wikipedia for the postcodes of Toronto and names of neighbourhoods & boroughs

In [123]:
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
soup = soup.find_all('td')
table_contents=[]
i=0
for cell in soup:
    i=i+1
    cellout = {}
    if hasattr(cell,'span'):
            if i > 180:
                #end of table - fix
                pass
            elif cell.span.text=='Not assigned':
                #passed
                pass
            else:
                #not passed
                cellout['PostalCode'] = cell.p.text[:3]
                cellout['Borough'] = (cell.span.text).split('(')[0]
                cellout['Neighborhood'] = (((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
                table_contents.append(cellout)

df1 = pd.DataFrame (table_contents,columns=["PostalCode","Borough","Neighborhood"])
df1['Borough']=df1['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
display(df1.head())
df1.shape

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


(103, 3)

In [124]:
!pip install geocoder
import geocoder # import geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [125]:
# DONT USE THIS - Google API doesn't respond
# SEE NOTES- WE ARE NOW USING GEOCODER INSTEAD AS GOOGLE CHARGES FEES
#
#    
#i=0
#for index, row in df1.iterrows():
#    i=i+1
#    # initialize your variable to None
#    lat_lng_coords = None
#
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(row["PostalCode"]))
#        lat_lng_coords = g.latlng
#    r["latitude"] = lat_lng_coords[0]
#    r["longitude"] = lat_lng_coords[1]
#    print(df1.head())
#    if i>2:
#        break
#    
#df1.head()



In [126]:
# DONT USE THIS - GEOCODER DOESNT RESPOND

#postal_code = "M5A"

# initialize your variable to None
#lat_lng_coords = None
#
## loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng
#
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

## We import neighbourhood lat & long data from CSV (other providers dont work withouth licenses)

In [127]:
import csv
import requests
df2=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
print(df2.head())
df1['Latitude']=''
df1['Longitude']=''
for i in df1.index:
    for j in df3.index:
        if(df1.at[i,"PostalCode"]==df3.at[j,"Postal Code"]):
            df1.at[i,"Latitude"]=df3.at[j,"Latitude"]
            df1.at[i,"Longitude"]=df3.at[j,"Longitude"]
                      
df1.head()

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Queen's Park,Ontario Provincial Government,43.6623,-79.3895


In [128]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


## Using Nominatim we identify the location of Toronto

In [129]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## We map all neighbourhoods on a Folium map

In [130]:
map_tr = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tr)  
    
map_tr

## We want to focus just on the Downtown borough - to learn what makes the business district tick. We filter by this borough and return neighbourhoods & locations, then find the central point of the Downtown borough

In [131]:
dtr_dat = df1[df1['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dtr_dat.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
2,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
3,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
4,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874


In [132]:
address = 'Downtown, Toronto'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081162653639.


## Now we mark Downtown neighbourhoods on a Folium map

In [133]:
map_dtr = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dtr_dat['Latitude'], dtr_dat['Longitude'], dtr_dat['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtr)  
    
map_dtr

## We set up Foursquare credentials and use it to return all venues (max 100) in the vicinity of Downtown

In [134]:
CLIENT_ID = 'ED51L2XHFB1IIKHKRCX3QDY2EXUOZ25SHHQV4JSTHFRECLH4' # your Foursquare ID
CLIENT_SECRET = 'T2BXC1HLDCLIY02VDN3NRTXK3DIAZWZY0KO1BJC0AV3FOPLA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ED51L2XHFB1IIKHKRCX3QDY2EXUOZ25SHHQV4JSTHFRECLH4
CLIENT_SECRET:T2BXC1HLDCLIY02VDN3NRTXK3DIAZWZY0KO1BJC0AV3FOPLA


In [135]:
dtr_dat.loc[0, 'Neighborhood']
nlat = dtr_dat.loc[0, 'Latitude'] # neighborhood latitude value
nlon = dtr_dat.loc[0, 'Longitude'] # neighborhood longitude value

nbr =dtr_dat.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(nbr,nlat,nlon))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [136]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nlat, 
    nlon, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ED51L2XHFB1IIKHKRCX3QDY2EXUOZ25SHHQV4JSTHFRECLH4&client_secret=T2BXC1HLDCLIY02VDN3NRTXK3DIAZWZY0KO1BJC0AV3FOPLA&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [137]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d88d5a0a93c65c3d2d26fc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

## We create a tidy dataframe of venue names, categories and locations within the Downdown borough

In [138]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [139]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

42 venues were returned by Foursquare.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [140]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [141]:
dtrvens = getNearbyVenues(names=dtr_dat['Neighborhood'],latitudes=dtr_dat['Latitude'],longitudes=dtr_dat['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [142]:
print(dtrvens.shape)
print(dtrvens.head())
print('There are {} uniques categories.'.format(len(dtrvens['Venue Category'].unique())))
dtrvens.groupby('Neighborhood').count()

(1067, 7)
                Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0           Tandem Coffee       43.653559       -79.361809   
1        Roselle Desserts       43.653447       -79.362017   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4          Impact Kitchen       43.656369       -79.356980   

        Venue Category  
0          Coffee Shop  
1               Bakery  
2  Distribution Center  
3                  Spa  
4           Resta

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,47,47,47,47,47,47
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,62,62,62,62,62,62
Christie,15,15,15,15,15,15
Church and Wellesley,67,67,67,67,67,67
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


## We use One Hot encoding to return the below matrix to be used for venue intensity calculations

In [143]:
# one hot encoding
tr_onehot = pd.get_dummies(dtrvens[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tr_onehot['Neighborhood'] = dtrvens['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tr_onehot.columns[-1]] + list(tr_onehot.columns[:-1])
tr_onehot = tr_onehot[fixed_columns]
print(tr_onehot.shape)
tr_onehot.head()


(1067, 183)


,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Taking the means, we now work out the intensity of each category by neighbourhood

In [144]:
trgroup = tr_onehot.groupby('Neighborhood').mean().reset_index()
print(trgroup.shape)
trgroup

(17, 183)


,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.042553,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.016129,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.016129,0.000000,0.000000,0.016129,0.000000
3,Christie,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.014925,0.014925,0.014925,0.0000,0.0000,0.000,0.000,0.000,0.014925,...,0.014925,0.014925,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.014925
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.020000,...,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.020000,...,0.000000,0.000000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.020000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.010000,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.016949,0.00,0.00,0.050847,0.000000,0.050847,0.016949,0.000000


## Now we can work out the top 5 venue categories in each neighbourhood

In [145]:
num_top_venues = 5

for hood in trgroup['Neighborhood']:
    print("----"+hood+"----")
    temp = trgroup[trgroup['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0    Cocktail Bar  0.09
1     Coffee Shop  0.06
2          Bakery  0.06
3  Sandwich Place  0.06
4        Beer Bar  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3               Bar  0.06
4             Plane  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.08
2    Sushi Restaurant  0.06
3  Italian Restaurant  0.05
4                Café  0.05


----Christie----
                venue  freq
0       Grocery Store  0.27
1                Café  0.20
2                Park  0.13
3          Baby Store  0.07
4  Italian Restaurant  0.07


----Church and Wellesley----
                 venue  freq
0     Sushi Restaurant  0.09
1  Japanese Restaurant  0.07
2          Coffee Shop  0.04
3              Gay Bar  0.04
4  

In [146]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## And finally we can work out the most common venue types by neighbourhood in a neat dataframe

In [147]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = trgroup['Neighborhood']

for ind in np.arange(trgroup.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(trgroup.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Sandwich Place,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Vegetarian / Vegan Restaurant,Liquor Store,Bistro
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Sculpture Garden,Boutique,Rental Car Location,Bar,Plane
2,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bank,Pizza Place,Burger Joint
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Restaurant,Italian Restaurant,Baby Store,Coffee Shop,Fried Chicken Joint
4,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Gay Bar,Men's Store,Mediterranean Restaurant,Burrito Place,Indian Restaurant,Pub


## We now want to set up clustering. Given Downtown Toronto is a relatively smaller place than Manhattan, we'll use only 5 clusters here

In [148]:
# set number of clusters
kclusters = 5

trg_cluster = trgroup.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(trg_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 4, 3, 0, 4, 4, 4, 4, 0], dtype=int32)

In [149]:
# add clustering labels
neighborhoods_venues_sorted["Cluster Labels"] = kmeans.labels_


## Now we merge the cluster and venue data back into our original neighbourhoods dataframe - and can interpret and map our findings

In [150]:
trmerge = dtr_dat

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
trmerge = trmerge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

trmerge.head(10) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,Coffee Shop,Park,Bakery,Pub,Café,Farmers Market,Beer Store,Electronics Store,Distribution Center,Restaurant,4
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,Coffee Shop,Clothing Store,Sandwich Place,Café,Pizza Place,Cosmetics Shop,Hotel,Japanese Restaurant,Bank,Italian Restaurant,4
2,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,Coffee Shop,Café,Italian Restaurant,Cocktail Bar,Restaurant,Clothing Store,Beer Bar,Gastropub,Farmers Market,Diner,0
3,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733,Cocktail Bar,Coffee Shop,Sandwich Place,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Vegetarian / Vegan Restaurant,Liquor Store,Bistro,0
4,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bank,Pizza Place,Burger Joint,4
5,M6G,Downtown Toronto,Christie,43.6695,-79.4226,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Restaurant,Italian Restaurant,Baby Store,Coffee Shop,Fried Chicken Joint,3
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6506,-79.3846,Coffee Shop,Café,Sandwich Place,Gym,Clothing Store,Sushi Restaurant,Restaurant,Bank,Bakery,Burrito Place,4
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.6408,-79.3818,Coffee Shop,Café,Hotel,Pizza Place,Scenic Lookout,Aquarium,Gym,Sports Bar,Brewery,Park,4
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.6472,-79.3816,Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Pharmacy,Restaurant,Salad Place,Deli / Bodega,Bakery,4
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.6482,-79.3798,Coffee Shop,Sandwich Place,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Bank,4


## We see two major clusters in the central downtown borough, and some individual neighbourhoods in the North, North West and at the airprort in the South which are less simiar to the core downtown neighbourhood

In [151]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(trmerge['Latitude'],trmerge['Longitude'],trmerge['Neighborhood'], trmerge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Our first cluster has four neighbourhoods, spread around the periphery of the core CBD - with a variety of restaurants, bars and coffee shops

In [152]:
trmerge.loc[trmerge['Cluster Labels'] == 0, trmerge.columns[[1] + list(range(5, trmerge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Downtown Toronto,Coffee Shop,Café,Italian Restaurant,Cocktail Bar,Restaurant,Clothing Store,Beer Bar,Gastropub,Farmers Market,Diner,0
3,Downtown Toronto,Cocktail Bar,Coffee Shop,Sandwich Place,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Vegetarian / Vegan Restaurant,Liquor Store,Bistro,0
11,Downtown Toronto,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Dessert Shop,Mexican Restaurant,Burger Joint,Bar,Gaming Cafe,0
16,Downtown Toronto,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Gay Bar,Men's Store,Mediterranean Restaurant,Burrito Place,Indian Restaurant,Pub,0


## The second cluster has more green space and recreational activities - north of the CBD

In [153]:
trmerge.loc[trmerge['Cluster Labels'] == 1, trmerge.columns[[1] + list(range(5, trmerge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
13,Downtown Toronto,Park,Trail,Playground,Concert Hall,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,1


## Our third cluster is just one neighbourhood, focused on the airport to the south, with businesses focused on servicing the airport's needs

In [154]:
trmerge.loc[trmerge['Cluster Labels'] == 2, trmerge.columns[[1] + list(range(5, trmerge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
12,Downtown Toronto,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Sculpture Garden,Boutique,Rental Car Location,Bar,Plane,2


## Cluster four is to the west of the others, in a more residential part of the city, with a larger number of grocery stores and parks than the CBD

In [155]:
trmerge.loc[trmerge['Cluster Labels'] == 3, trmerge.columns[[1] + list(range(5, trmerge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
5,Downtown Toronto,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Restaurant,Italian Restaurant,Baby Store,Coffee Shop,Fried Chicken Joint,3


## Our final cluster is the core CBD district, with an extremely heavy focus on cafes and coffee shops, as well as hotels and gyms - servicing the needs of users of the central downtown part of the city

In [156]:
trmerge.loc[trmerge['Cluster Labels'] == 4, trmerge.columns[[1] + list(range(5, trmerge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Downtown Toronto,Coffee Shop,Park,Bakery,Pub,Café,Farmers Market,Beer Store,Electronics Store,Distribution Center,Restaurant,4
1,Downtown Toronto,Coffee Shop,Clothing Store,Sandwich Place,Café,Pizza Place,Cosmetics Shop,Hotel,Japanese Restaurant,Bank,Italian Restaurant,4
4,Downtown Toronto,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bank,Pizza Place,Burger Joint,4
6,Downtown Toronto,Coffee Shop,Café,Sandwich Place,Gym,Clothing Store,Sushi Restaurant,Restaurant,Bank,Bakery,Burrito Place,4
7,Downtown Toronto,Coffee Shop,Café,Hotel,Pizza Place,Scenic Lookout,Aquarium,Gym,Sports Bar,Brewery,Park,4
8,Downtown Toronto,Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Pharmacy,Restaurant,Salad Place,Deli / Bodega,Bakery,4
9,Downtown Toronto,Coffee Shop,Sandwich Place,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Bank,4
10,Downtown Toronto,Café,Coffee Shop,Sandwich Place,Bar,Japanese Restaurant,Bakery,Pub,Comfort Food Restaurant,Poutine Place,College Gym,4
14,Downtown Toronto,Coffee Shop,Café,Bakery,Restaurant,Pub,Italian Restaurant,Pizza Place,Chinese Restaurant,Beer Store,Pharmacy,4
15,Downtown Toronto,Coffee Shop,Sandwich Place,Café,Hotel,Japanese Restaurant,Bank,Gym,Deli / Bodega,Restaurant,Sushi Restaurant,4
